In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from transformers import BertTokenizer, BertModel
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertEncoder

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import wandb
from tqdm import tqdm
import os

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **1. Load the Data**

In [ ]:
!unzip "/content/drive/MyDrive/GENAI/Week7/Day4/llm-detect-ai-generated-text.zip"

In [6]:
file1 = pd.read_csv("/content/train_essays.csv")
df_train = pd.DataFrame(file1)
df_train.drop_duplicates(inplace=True)
df_train.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [7]:
df_train.shape

(1378, 4)

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB


In [9]:
df_train.isna().sum()

,0
id,0
prompt_id,0
text,0
generated,0


In [10]:
df_train["generated"].value_counts()

,count
generated,
0,1375
1,3


In [11]:
df_train[df_train["generated"] == 1]

,id,prompt_id,text,generated
704,82131f68,1,"This essay will analyze, discuss and prove one...",1
740,86fe4f18,1,I strongly believe that the Electoral College ...,1
1262,eafb8a56,0,"Limiting car use causes pollution, increases c...",1


Le dataset ne contient que 3 textes générés par IA contre 1375 textes ecrit pas des humains

# **2. Prepare the Model**

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
model.to(device)

In [ ]:
model.eval()

In [15]:
def embeddings(text):
  all_embeddings = []

  for i in range(0, len(text), 32):
    batch = text[i:i+32]
    encoded = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    with torch.no_grad():
      outputs = model(input_ids, attention_mask=attention_mask)
      cls_emb = outputs.last_hidden_state[:,0,:]
      all_embeddings.append(cls_emb.cpu())

  return torch.cat(all_embeddings, dim=0)

# **3. Set Hyperparameters**

In [98]:
train_batch_size = 16
test_batch_size = 16
latent_dim = 100
lr = 2e-5
beta1 = 0.5
beta2 = 0.999
num_epochs = 10
save = 1

# **4. Prepare the Data for Training**

In [17]:
df_human = df_train[df_train["generated"] == 0]
human_list = df_human["text"].tolist()
human_embeddings = embeddings(human_list)

In [18]:
type(human_embeddings)

torch.Tensor

In [19]:
labels_human = torch.ones(len(human_embeddings))

In [20]:
df_fake = df_train[df_train["generated"] == 1]
texts_fake = df_fake["text"].tolist()
embeddings_fake = embeddings(texts_fake)

In [21]:
labels_fake = torch.zeros(len(embeddings_fake))

In [22]:
X_train, X_human_val, y_train, y_human_val = train_test_split(
    human_embeddings, labels_human, test_size=0.2, random_state=42
)

In [23]:
X_val = torch.cat([X_human_val, embeddings_fake], dim=0)
y_val = torch.cat([y_human_val, labels_fake], dim=0)

In [24]:
train_dataset  = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

In [25]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# **6. Define the Generator Model**

In [88]:
config = BertConfig(
    hidden_size = 768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072
    )

class Generator(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.fc = nn.Linear(input_dim, 256 * 4)
    self.conv_net = nn.Sequential(
      nn.ConvTranspose1d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
      nn.BatchNorm1d(512),
      nn.ReLU(True),

      nn.ConvTranspose1d(512, 256, 4, 2, 1, bias=False),
      nn.BatchNorm1d(256),
      nn.ReLU(True),

      nn.ConvTranspose1d(256, 128, 4, 2, 1, bias=False),
      nn.BatchNorm1d(128),
      nn.ReLU(True),

      nn.ConvTranspose1d(128, 768, 4, 2, 1, bias=False),
      nn.Tanh()
    )
    self.bert_encoder = BertEncoder(config)
    self.max_length = 512
    self.position_embedding = nn.Parameter(
        torch.randn(1, self.max_length, 768) * 0.02
        )

  def forward(self, x):
    batch_size = x.size(0)

    x = self.fc(x)
    x = x.view(batch_size, 256, 4)
    x = self.conv_net(x)
    x = x.permute(0, 2, 1)
    seq_len = x.size(1)
    x = x + self.position_embedding[:, :seq_len, :]

    attention_mask = torch.ones((batch_size, seq_len), device=x.device, dtype=torch.long)
    extended_mask = attention_mask[:, None, None, :]
    extended_mask = (1.0 - extended_mask) * -10000.0
    encoded = self.bert_encoder(
        hidden_states=x,
        attention_mask=extended_mask
    )

    return encoded.last_hidden_state

# **7. Define the Discriminator Model**

In [89]:
class SumBertPooler(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, hidden_states):

    return hidden_states.sum(dim=1)

In [90]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_encoder = BertEncoder(config)
    self.bert_encoder.layer = nn.ModuleList([
        layer for layer in model.encoder.layer[:6]
    ])
    self.pooler = SumBertPooler()
    self.classifier = nn.Sequential(
       nn. Linear(768,256),
       nn.ReLU(),
       nn.Dropout(0.3),
       nn.Linear(256,1)
    )

  def forward(self, input):

    if input.dim() == 2:
      input = input.unsqueeze(1)

    batch_size = input.size(0)
    seq_len = input.size(1)

    attention_mask = torch.ones((batch_size, seq_len), device=input.device, dtype=torch.long)
    extended_mask = attention_mask[:, None, None, :]
    extended_mask = (1.0 - extended_mask) * -10000.0

    out = self.bert_encoder(hidden_states=input, attention_mask=extended_mask).last_hidden_state
    out = self.pooler(out)
    out = self.classifier(out)
    return out.view(-1)

# **8. Weights Initilization**

In [91]:
def weights_init(m):
  if isinstance(m, nn.Linear):
    nn.init.xavier_uniform_(m.weight)
    nn.init.zeros_(m.bias)
  elif isinstance(m, nn.ConvTranspose1d):
    nn.init.normal_(m.weight.data, 0.0, 0.02)

# **9. Train the Model**

In [30]:
!pip install --q wandb

In [ ]:
wandb.init(project="gan_classifier", name="Building a GAN-Based AI Text Detector")

generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

generator.apply(weights_init)
discriminator.apply(weights_init)

g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
  progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

  for step, (real_embeddings, _) in progress_bar:
    batch_size = real_embeddings.size(0)
    real_embeddings = real_embeddings.to(device)

    real_labels = torch.ones(batch_size, device=device)
    fake_labels = torch.zeros(batch_size, device=device)

    z = torch.randn(batch_size, latent_dim, device=device)
    fake_embeddings = generator(z).detach()

    real_output = discriminator(real_embeddings)
    fake_output = discriminator(fake_embeddings)

    d_loss_real = criterion(real_output, real_labels)
    d_loss_fake = criterion(fake_output, fake_labels)
    d_loss = d_loss_real + d_loss_fake

    d_optimizer.zero_grad()
    d_loss.backward()
    nn.utils.clip_grad_norm_(discriminator.parameters(), 1.0)
    d_optimizer.step()

    z = torch.randn(batch_size, latent_dim, device=device)
    generated = generator(z)
    predictions = discriminator(generated)
    g_loss = criterion(predictions, real_labels)

    g_optimizer.zero_grad()
    g_loss.backward()
    nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
    g_optimizer.step()

    wandb.log({
        "epoch": epoch + 1,
        "d_loss": d_loss.item(),
        "g_loss": g_loss.item(),
    })

  generator.eval()
  discriminator.eval()
  all_preds = []
  all_labels = []

  with torch.no_grad():
    for val_embeddings, val_labels in val_loader:
      val_embeddings = val_embeddings.to(device)
      val_labels = val_labels.to(device)
      outputs = discriminator(val_embeddings)
      val_loss = criterion(outputs, val_labels)
      all_preds.extend(outputs.tolist())
      all_labels.extend(val_labels.tolist())

    auc_score = roc_auc_score(all_labels, all_preds)
    wandb.log({"AUC":auc_score})

  torch.save(generator.state_dict(), f"/content/drive/MyDrive/GENAI/Week7/Day4/gan_generator/generator3_epoch_{epoch+1}.pth")
  torch.save(discriminator.state_dict(), f"/content/drive/MyDrive/GENAI/Week7/Day4/discriminator3_epoch_{epoch+1}.pth")

In [ ]:
path_1 = "/content/drive/MyDrive/GENAI/Week7/Day4/gan_generator/generator3_epoch_10.pth"
path_2 = "/content/drive/MyDrive/GENAI/Week7/Day4/gan_discriminator/discriminator3_epoch_10.pth"

generator = Generator(latent_dim).to(device)
generator.load_state_dict(torch.load(path_1, map_location=device))
generator.eval()

discriminator = Discriminator().to(device)
discriminator.load_state_dict(torch.load(path_2, map_location=device))
generator.eval()